In [2]:
import pandas as pd
import numpy as np
import os
import pymysql as ps


In [148]:
p_df = pd.read_csv(os.getcwd() + os.sep + 'p_info.csv', engine='python')

In [149]:
i = 0
sex_list = ['남', '여']
sex_result_list = []
for i in range(len(p_df)): 
  flag = np.random.randint(0,2,1)
  sex_result_list.append(sex_list[flag[0]]) 

i = 0
fallen_list = ['예', '아니오']
fallen_result_list = []
for i in range(len(p_df)): 
  flag = np.random.randint(0,2,1)
  fallen_result_list.append(fallen_list[flag[0]])

i = 0
bedsore_list = ['예', '아니오']
bedsore_result_list = []
for i in range(len(p_df)): 
  flag = np.random.randint(0,2,1)
  bedsore_result_list.append(bedsore_list[flag[0]])  

i = 0
stat_list = ['침대 위', '침대 밖']
stat_result_list = []
for i in range(len(p_df)): 
  flag = np.random.randint(0,2,1)
  stat_result_list.append(stat_list[flag[0]])


In [150]:
p_df['p_id'] = p_df['p_id'].fillna(pd.Series(np.random.randint(1000000, 3000000, len(p_df))))
p_df['p_id'] = p_df['p_id'].astype(np.int64)

p_df['age'] = p_df['age'].fillna(pd.Series(np.random.randint(15, 70, len(p_df))))
p_df['age'] = p_df['age'].astype(np.int64)

p_df['sex'] = p_df['sex'].fillna(pd.Series(sex_result_list))
p_df['fallen'] = p_df['fallen'].fillna(pd.Series(fallen_result_list))
p_df['bedsore'] = p_df['bedsore'].fillna(pd.Series(bedsore_result_list))
p_df['status'] = p_df['status'].fillna(pd.Series(stat_result_list))

p_df.to_csv(os.getcwd() + os.sep + "p_info2.csv", index=False, encoding="utf-8-sig")

In [2]:
p_df = pd.read_csv(os.getcwd() + os.sep + "p_info2.csv", encoding="utf-8-sig")

In [3]:
"""DataBase 정보"""
sql_args = {
    'host' : '127.0.0.1',   ## local host
    'port' : 3306,          ## local port
    'user' : 'admin',       ## Database ID
    'password' : 'tkdgns',  ## Database Password
    'db' : 'bass_test'      ## DB name
}

In [4]:
total_p_df = p_df

room_1_p_df = p_df[p_df['no_room'] == 1]
room_1_p_df = room_1_p_df.reset_index()

room_2_p_df = p_df[p_df['no_room'] == 2]
room_2_p_df = room_2_p_df.reset_index()

room_3_p_df = p_df[p_df['no_room'] == 3]
room_3_p_df = room_3_p_df.reset_index()

room_4_p_df = p_df[p_df['no_room'] == 4]
room_4_p_df = room_4_p_df.reset_index()

In [5]:
df_list = [total_p_df, room_1_p_df, room_2_p_df, room_3_p_df, room_4_p_df]
df_name_list = ['total_p', 'room_1_p', 'room_2_p',
                'room_3_p', 'room_4_p']

In [6]:
# 초기값 설정
conn = None
cursor = None

sql = ""

# DB Connection
db = ps.connect(host = sql_args['host'], port = sql_args['port'], user = sql_args['user'], 
                password = sql_args['password'], db = sql_args['db'], charset = 'utf8')

# Cursor 객체 생성
cursor = db.cursor(ps.cursors.DictCursor)

i = 0
for i in range(len(df_name_list)):
  # 특정 테이블 조회
  table_name = df_name_list[i]

  check="show tables like '{}';".format(table_name)


  # Cursor를 이용하여 쿼리문 실행
  cursor.execute(check)

  result = cursor.fetchall()

  if len(result) == 0:
      
      print("{} 테이블 없음.".format(table_name))
      sql = '''create table {}(
            p_id varchar(20) not null,
            p_name varchar(20) not null,
            no_ward varchar(20) not null,
            no_room varchar(20) not null,
            no_bed varchar(20) not null,
            age varchar(20) not null,
            sex varchar(20) not null,
            fallen varchar(20) not null,
            bedsore varchar(20) not null,
            status varchar(20) not null,
            primary key(p_id, p_name)
            );
            '''.format(table_name)
      cursor.execute(sql)
      db.commit()
      print("{} 테이블 생성완료.".format(table_name))

i = 0
sub_list = []
for i in range(len(df_list)):
  df = df_list[i]
  table_name = df_name_list[i]
  j = 0
  for j in range(len(df)):
    p_id = df['p_id'][j]
    p_name = df['p_name'][j]
    no_ward = df['no_ward'][j]
    no_room = df['no_room'][j]
    no_bed = df['no_bed'][j]
    age = df['age'][j]
    sex = df['sex'][j]
    fallen = df['fallen'][j]
    bedsore = df['bedsore'][j]
    status = df['status'][j]

    insert_sql = "SELECT * FROM {} WHERE {} = %s AND {} = %s;".format(table_name,
                                                                      'p_id',
                                                                      'p_name')
    cursor.execute(insert_sql, (p_id, p_name))

    result = cursor.fetchall()
    print(result)
    if result == ():                                                               
        sub = (p_id, p_name, no_ward, no_room, no_bed,
               age, sex, fallen, bedsore, status)

        sub_list.append(sub)
        print()
        insert_sql = "INSERT INTO {} VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);".format(table_name)
        cursor.execute(insert_sql, (p_id, p_name, no_ward, no_room, no_bed,
                                    age, sex, fallen, bedsore, status))
        print('{} 테이블 {} 입력 완료'.format(table_name, sub_list))
        
    sub_list = []    

db.commit()

total_p 테이블 없음.
total_p 테이블 생성완료.
room_1_p 테이블 없음.
room_1_p 테이블 생성완료.
room_2_p 테이블 없음.
room_2_p 테이블 생성완료.
room_3_p 테이블 없음.
room_3_p 테이블 생성완료.
room_4_p 테이블 없음.
room_4_p 테이블 생성완료.
()

total_p 테이블 [(2340287, '김아무개', 1, 1, 1, 22, '여', '예', '아니오', '침대 위')] 입력 완료
()

total_p 테이블 [(1886749, '이아무개', 1, 1, 2, 23, '남', '아니오', '아니오', '침대 밖')] 입력 완료
()

total_p 테이블 [(2742259, '박아무개', 1, 1, 3, 62, '여', '예', '예', '침대 밖')] 입력 완료
()

total_p 테이블 [(1946504, '최아무개', 1, 1, 4, 44, '남', '아니오', '아니오', '침대 위')] 입력 완료
()

total_p 테이블 [(1118016, '정아무개', 1, 1, 5, 38, '남', '아니오', '예', '침대 위')] 입력 완료
()

total_p 테이블 [(1265709, '강아무개', 1, 1, 6, 24, '남', '아니오', '예', '침대 위')] 입력 완료
()

total_p 테이블 [(1539996, '조아무개', 1, 1, 7, 56, '여', '아니오', '예', '침대 위')] 입력 완료
()

total_p 테이블 [(1133932, '윤아무개', 1, 1, 8, 52, '여', '예', '예', '침대 밖')] 입력 완료
()

total_p 테이블 [(2986538, '장아무개', 1, 1, 9, 46, '남', '아니오', '예', '침대 위')] 입력 완료
()

total_p 테이블 [(1039232, '임아무개', 1, 1, 10, 26, '여', '아니오', '예', '침대 밖')] 입력 완료
()

total_p 테이블 [(111

In [3]:
p_df = pd.read_csv(os.getcwd() + os.sep + "p_info2.csv", encoding="utf-8-sig")

In [19]:
room_option_name = '1호실'
patient_option_name = '전체 환자'

result = csv_to_py(p_df, room_option_name, patient_option_name)

In [20]:
result

,index,p_id,p_name,no_ward,no_room,no_bed,age,sex,fallen,bedsore,status
0,0,2340287,김아무개,1,1,1,22,여,예,아니오,침대 위
1,1,1886749,이아무개,1,1,2,23,남,아니오,아니오,침대 밖
2,2,2742259,박아무개,1,1,3,62,여,예,예,침대 밖
3,3,1946504,최아무개,1,1,4,44,남,아니오,아니오,침대 위
4,4,1118016,정아무개,1,1,5,38,남,아니오,예,침대 위
5,5,1265709,강아무개,1,1,6,24,남,아니오,예,침대 위
6,6,1539996,조아무개,1,1,7,56,여,아니오,예,침대 위
7,7,1133932,윤아무개,1,1,8,52,여,예,예,침대 밖
8,8,2986538,장아무개,1,1,9,46,남,아니오,예,침대 위
9,9,1039232,임아무개,1,1,10,26,여,아니오,예,침대 밖


In [18]:
def csv_to_py(p_df, room_option_name, patient_option_name):
    
    if room_option_name == '전체호실':
      p_df = p_df

    elif room_option_name == '1호실':
      p_df = p_df[p_df['no_room'] == 1]
      p_df = p_df.reset_index()

    elif room_option_name == '2호실':
      p_df = p_df[p_df['no_room'] == 2]
      p_df = p_df.reset_index()

    elif room_option_name == '3호실':
      p_df = p_df[p_df['no_room'] == 3]
      p_df = p_df.reset_index()

    elif room_option_name == '4호실':
      p_df = p_df[p_df['no_room'] == 4]
      p_df = p_df.reset_index()

    if patient_option_name == '전체 환자':
      result = p_df

    elif patient_option_name == '낙상 주의 환자':
      result = p_df[p_df['fallen'] == '예']

    elif patient_option_name == '욕창 주의 환자':  
      result = p_df[p_df['bedsore'] == '예']
    
    return result

In [5]:
total_p_df = p_df

room_1_p_df = p_df[p_df['no_room'] == 1]
room_1_p_df = room_1_p_df.reset_index()

room_2_p_df = p_df[p_df['no_room'] == 2]
room_2_p_df = room_2_p_df.reset_index()

room_3_p_df = p_df[p_df['no_room'] == 3]
room_3_p_df = room_3_p_df.reset_index()

room_4_p_df = p_df[p_df['no_room'] == 4]
room_4_p_df = room_4_p_df.reset_index()

In [7]:
room_1_p_df_1 = room_1_p_df[room_1_p_df['fallen'] == '예']

In [8]:
room_1_p_df_1

,index,p_id,p_name,no_ward,no_room,no_bed,age,sex,fallen,bedsore,status
0,0,2340287,김아무개,1,1,1,22,여,예,아니오,침대 위
2,2,2742259,박아무개,1,1,3,62,여,예,예,침대 밖
7,7,1133932,윤아무개,1,1,8,52,여,예,예,침대 밖


In [9]:
result = total_p_df

In [14]:
result.loc[0]

p_id       2340287
p_name        김아무개
no_ward          1
no_room          1
no_bed           1
age             22
sex              여
fallen           예
bedsore        아니오
status        침대 위
Name: 0, dtype: object

In [16]:
result_args = result.loc[0]

In [17]:
result_args['p_id']

2340287